In [3]:
import subprocess
import sys
import json
import os
import logging
import pandas as pd
from jira import JIRA
from datetime import datetime

In [1]:
# Função para configurar logging dinamicamente
def setup_logging(log_file_path):
    import logging as logging_mod
    handler = logging_mod.FileHandler(log_file_path)
    formatter = logging_mod.Formatter('%(asctime)s - %(levelname)s - %(message)s')
    handler.setFormatter(formatter)
    logger = logging_mod.getLogger()
    logger.setLevel(logging_mod.INFO)
    logger.addHandler(handler)
    return logger

# Função para instalar pacotes automaticamente se não estiverem presentes
def install_and_import(package):
    try:
        __import__(package)
    except ImportError:
        logging.warning(f"Pacote '{package}' não encontrado. Instalando...")
        try:
            subprocess.check_call([sys.executable, "-m", "pip", "install", package])
            logging.info(f"Pacote '{package}' instalado com sucesso!")
        except subprocess.CalledProcessError:
            logging.error(f"Erro ao tentar instalar o pacote '{package}'. Verifique a conexão ou permissões.")
            sys.exit(1)

# Função para carregar configurações do arquivo JSON
def load_config(config_file='jira_configs.json'):
    try:
        with open(config_file, 'r') as file:
            return json.load(file)
    except FileNotFoundError:
        logging.error(f"Arquivo de configuração '{config_file}' não encontrado.")
        sys.exit(1)
    except json.JSONDecodeError:
        logging.error(f"Erro ao ler o arquivo de configuração '{config_file}'. Verifique o formato JSON.")
        sys.exit(1)

# Instalar/importar pacotes necessários
install_and_import('pandas')
install_and_import('jira')


In [ ]:
# Função para carregar configurações do arquivo JSON
def load_config(config_file='configs/jira_configs.json'):
    if not os.path.exists(config_file):
        print(f"Arquivo de configuração '{config_file}' não encontrado. Criando arquivo com configurações padrão.")
        # Garantir que o diretório existe
        os.makedirs(os.path.dirname(config_file), exist_ok=True)
        default_config = {
            "JIRA_SERVER": "https://your_jira_server",
            "JIRA_USER": "your_username",
            "JIRA_TOKEN": "your_token",
            "PROJECT_KEY": "your_project_key",
            "FOLDER_LOGS": "."
        }
        with open(config_file, 'w') as file:
            json.dump(default_config, file, indent=4)
        return default_config
    try:
        with open(config_file, 'r') as file:
            return json.load(file)
    except json.JSONDecodeError:
        print(f"Erro ao ler o arquivo de configuração '{config_file}'. Verifique o formato JSON.")
        raise

# Atribuição de variáveis de configuração
JIRA_SERVER = config['JIRA_SERVER']
JIRA_USER = config['JIRA_USER']
JIRA_TOKEN = config['JIRA_TOKEN']
PROJECT_KEY = config['PROJECT_KEY']
folder_logs_path = config['FOLDER_LOGS']
folder_reports_path = config['FOLDER_REPORTS']

NameError: name 'os' is not defined

In [ ]:
# Gerar nome dinâmico para o arquivo de log com timestamp
timestamp = datetime.now().strftime('%Y-%m-%d_%H-%M-%S')
log_dir = os.path.dirname(folder_logs_path)
log_file_path = os.path.join(log_dir, f"jira_creation_log_{timestamp}.log")

# Configuração de logging
logger = setup_logging(log_file_path)
logger.info(f"Log gerado em: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")


In [ ]:
# Conexão com o Jira
options = {'server': JIRA_SERVER}
jira = JIRA(options, basic_auth=(JIRA_USER, JIRA_TOKEN))

In [ ]:
# Função para obter um EPIC existente
def get_existing_epic(epic_name):
    jql = f'project = {PROJECT_KEY} AND summary ~ "{epic_name}" AND issuetype = "Epic"'
    existing_epics = jira.search_issues(jql)
    return existing_epics[0] if existing_epics else None

# Função para criar um EPIC
def create_epic(epic_name):
    existing_epic = get_existing_epic(epic_name)
    if existing_epic:
        logging.info(f"EPIC '{epic_name}' já existe. Pulando criação.")
        return existing_epic
    else:
        issue_dict = {
            'project': {'key': PROJECT_KEY},
            'summary': epic_name,
            'description': f"EPIC criado automaticamente: {epic_name}",
            'issuetype': {'id': '10006'}  # ID do Epic
        }
        new_epic = jira.create_issue(fields=issue_dict)
        logging.info(f"EPIC '{epic_name}' criado com sucesso!")
        return new_epic


In [ ]:
# Função para verificar se o card já existe
def get_existing_card(summary):
    jql = f'project = {PROJECT_KEY} AND summary ~ "{summary}" AND issuetype = "10005"'  # ID de "Tarefa"
    existing_cards = jira.search_issues(jql)
    return existing_cards[0] if existing_cards else None

# Função para criar um card vinculado a um EPIC
def create_card(epic, issue_name, duration):
    existing_card = get_existing_card(issue_name)
    if existing_card:
        logging.info(f"Card '{issue_name}' já existe. Pulando criação.")
        return existing_card
    else:
        issue_dict = {
            'project': {'key': PROJECT_KEY},
            'summary': issue_name,
            'description': f"Card criado automaticamente: {issue_name} ({duration})",
            'issuetype': {'id': '10005'},  # ID de "Tarefa"
            'parent': {'key': epic.key}
        }
        new_card = jira.create_issue(fields=issue_dict)
        logging.info(f"Card '{issue_name}' criado com sucesso!")
        return new_card

In [ ]:
# Leitura do arquivo CSV com pandas
csv_file = os.path.join(folder_reports_path, 'project_specs_jira.csv')
df = pd.read_csv(csv_file, delimiter=';')

# Formatar todos os textos para MAIÚSCULAS
df['EPIC Name'] = df['EPIC Name'].str.upper()
df['Issue'] = df['Issue'].str.upper()
df['Duration Issue'] = df['Duration Issue'].str.upper()


In [ ]:
# Processar o CSV e criar EPICs e cards
for _, row in df.iterrows():
    epic_name = row['EPIC Name']
    issue_name = row['Issue']
    duration = row['Duration Issue']
    
    # Criação do EPIC
    epic = create_epic(epic_name)
    
    # Criação do card
    create_card(epic, issue_name, duration)

logging.info("Processo concluído com sucesso!")
print("Processo concluído com sucesso!")

Processo concluído com sucesso!
